In [35]:
import cv2
import os
import sys
from string import Template
from PIL import Image
import numpy as np

In [13]:
cv2.__version__

'3.1.0'

In [18]:
def get_im_list(im_dir):
    os.chdir(im_dir)
    imlist = []
    allfiles=os.listdir(im_dir)
    for filename in allfiles:
        if filename[-4:] in [".png",".PNG"] or filename[-4:] in [".jpg",".JPG"] or filename[-5:] in [".jpeg",".JPEG"]:
            imlist.append(filename)
    return imlist

In [53]:
def get_facial_translations(imlist):
    
    face_cascade = cv2.CascadeClassifier('/Users/noon/Desktop/new media 190/Fan-Art-Project/haarcascade_frontalface_default.xml')

    scale_factor = 1.2
    min_neighbors = 3
    min_size = (30, 30)
    flags = cv2.CASCADE_SCALE_IMAGE
    translations = []
    for infname in imlist:
        image_path = os.path.expanduser(infname)
        image = cv2.imread(image_path)

        faces = face_cascade.detectMultiScale(image, scaleFactor = scale_factor, minNeighbors = min_neighbors,
        minSize = min_size, flags = flags)
        print 'faces for im ', infname, ' are ', faces
        if len(faces) == 0:
            translations.append([0,0])
        else:
            face = faces[0]
            x = face[0]
            y = face[1]
            w = face[2]
            h = face[3]
            translations.append([int(x + w/2), int(y + h/2)])
    return translations

In [30]:
# first argument is the haarcascades path

face_cascade = cv2.CascadeClassifier('/Users/noon/Desktop/new media 190/Fan-Art-Project/haarcascade_frontalface_default.xml')

scale_factor = 1.2
min_neighbors = 3
min_size = (30, 30)
flags = cv2.CASCADE_SCALE_IMAGE

imdir = '/Users/noon/desktop/new media 190/fanart/data/hermione/after'
imlist = get_im_list(imdir)
translations = []
for infname in imlist:
    image_path = os.path.expanduser(infname)
    image = cv2.imread(image_path)

    faces = face_cascade.detectMultiScale(image, scaleFactor = scale_factor, minNeighbors = min_neighbors,
    minSize = min_size, flags = flags)
    print 'faces for im ', infname, ' are ', faces
    if len(faces) == 0:
        translations.append([0,0])
    else:
        face = faces[0]
        x = face[0]
        y = face[1]
        w = face[2]
        h = face[3]
        translations.append([int(x + w/2), int(y + h/2)])
print 'final translations are ', translations

faces for im  tumblr_nzpv2lFF3y1u4r6qwo1_1280.jpg  are  ()
faces for im  tumblr_nzq5vxadGf1u2bhf7o1_1280.png  are  [[257 196 404 404]]
faces for im  tumblr_nzq8k6f08t1s4uf7yo1_500.png  are  [[124 144 201 201]]
faces for im  tumblr_nzq9o6RvO61s01qsxo1_1280.png  are  ()
faces for im  tumblr_nzqb83khkS1qfmp5ko1_500.jpg  are  [[189  81 120 120]
 [ 69  88 295 295]]
faces for im  tumblr_nzqfy4OALi1uqa9pho1_1280.jpg  are  [[110 172  76  76]]
faces for im  tumblr_nzqr6xMMvK1ulk09ro1_1280.jpg  are  ()
faces for im  tumblr_nzr4r0jvOH1tkmijro1_1280.png  are  [[367 167 189 189]]
faces for im  tumblr_nzrbjpg8QZ1ry2v8mo1_1280.jpg  are  [[159 220 273 273]]
faces for im  tumblr_nzrtatDAA21unurdwo1_1280.jpg  are  [[338 396  61  61]
 [184 250  72  72]
 [489  38  54  54]
 [251  49  82  82]
 [ 30  21 162 162]
 [328 436  69  69]]
faces for im  tumblr_nzrw6wExOH1ubicsjo1_1280.jpg  are  [[274 426 116 116]]
faces for im  tumblr_nzrwl2hb6r1rj1knno1_1280.jpg  are  [[322 202 432 432]]
faces for im  tumblr_nzs0bl

In [47]:
def get_avg_im(imlist, fname, translations):
    #translation is list of lists [[x,y][x,y]...]
    max_w = max(Image.open(im).size[0] for im in imlist)
    max_h = max(Image.open(im).size[1] for im in imlist)
    max_x = max(x[0] for x in translations)
    max_y = max(x[1] for x in translations)

    print 'max w, h: ', max_w, max_h, max_x, max_y
    N=len(imlist)

    # Create a numpy array of floats to store the average (assume RGB images)
    arr=np.zeros((max_h+max_y,max_w+max_x,3),np.float)

    # Build up average pixel intensities, casting each image as an array of floats
    for im, xform in zip(imlist, translations):
        print 'working on ' + im
        old_im = Image.open(im)
        old_size = old_im.size
        new_size = (max_w+max_x, max_h+max_y)

        new_im = Image.new("RGB", new_size) 
        new_im.paste(old_im, ((new_size[0]-old_size[0])/2 + xform[0],
                              (new_size[1]-old_size[1])/2 + xform[1]))
        imarr=np.array(new_im,dtype=np.float)
        arr=arr+imarr/N
    # Round values in array and cast as 8-bit integer
    arr=np.array(np.round(arr),dtype=np.uint8)

    # Generate, save and preview final image
    out=Image.fromarray(arr,mode="RGB")
    out.save(fname+"-average.png")
    out.show()

In [48]:
get_avg_im(imlist, 'hermione', translations)

max w, h:  1280 1920 1002 1213
working on tumblr_nzpv2lFF3y1u4r6qwo1_1280.jpg
working on tumblr_nzq5vxadGf1u2bhf7o1_1280.png
working on tumblr_nzq8k6f08t1s4uf7yo1_500.png
working on tumblr_nzq9o6RvO61s01qsxo1_1280.png
working on tumblr_nzqb83khkS1qfmp5ko1_500.jpg
working on tumblr_nzqfy4OALi1uqa9pho1_1280.jpg
working on tumblr_nzqr6xMMvK1ulk09ro1_1280.jpg
working on tumblr_nzr4r0jvOH1tkmijro1_1280.png
working on tumblr_nzrbjpg8QZ1ry2v8mo1_1280.jpg
working on tumblr_nzrtatDAA21unurdwo1_1280.jpg
working on tumblr_nzrw6wExOH1ubicsjo1_1280.jpg
working on tumblr_nzrwl2hb6r1rj1knno1_1280.jpg
working on tumblr_nzs0blryq41ulk7bxo1_400.jpg
working on tumblr_nzs4nqNyCG1td7ez0o1_1280.jpg
working on tumblr_nzs4wbtKNQ1r001pfo1_1280.jpg
working on tumblr_nzs5f0fTw11uv3dfqo1_1280.jpg
working on tumblr_nzsj20ZIwH1r5a236o1_400.png
working on tumblr_nzskn0rXof1uxk2c3o1_500.jpg
working on tumblr_nztdv7uide1qd1pu0o1_1280.png
working on tumblr_nzu2hkPuEs1u5yvtmo1_r1_1280.png
working on tumblr_nzufzpH80t1qz

In [ ]:
d = '/Users/noon/Desktop/new media 190/fanart/data/'
dirs = filter(lambda x: os.path.isdir(os.path.join(d, x)), os.listdir(d))
print dirs
for fandom in dirs:
    if fandom == 'unused': continue
    before = os.path.join(d, fandom) + '/before'
    after = os.path.join(d, fandom) + '/after'
    imlist_before = get_im_list(before)
    xform_before = get_facial_translations(imlist_before)
    get_avg_im(imlist_before, fandom+'-before', xform_before)
    imlist_after = get_im_list(after)
    xform_after = get_facial_translations(imlist_after)
    get_avg_im(imlist_after, fandom+'-after', xform_after)

['clintasha', 'finn', 'hannigram', 'hermione', 'johnlock', 'korrasami', 'unused']
faces for im  tumblr_nmmf326OZf1spz493o1_250.jpg  are  ()
faces for im  tumblr_nmohfoyrpy1tnli5ho1_1280.jpg  are  ()
faces for im  tumblr_nmpuov7ELE1qh1a6vo1_1280.png  are  [[389 160  86  86]]
faces for im  tumblr_nmsqna3fyi1rrrgqdo1_r1_540.jpg  are  [[312  32  58  58]
 [149  38  58  58]]
faces for im  tumblr_nmtsl0ST8B1qi629bo1_1280.png  are  ()
faces for im  tumblr_nmu0b7yh2S1rewfsco1_r5_1280.png  are  ()
faces for im  tumblr_nmuqp4VZ8o1s7vizso1_1280.jpg  are  ()
faces for im  tumblr_nmxum51jmL1qi629bo1_500.png  are  [[275  62  81  81]]
faces for im  tumblr_nn056xwzZ01qbzkl6o1_540.jpg  are  [[322 286 205 205]]
faces for im  tumblr_nn1krlHuI61qi629bo1_1280.png  are  [[139 150  84  84]
 [407 163  74  74]]
faces for im  tumblr_nn4r5tTzhB1qdu1txo1_1280.jpg  are  [[338 280  72  72]]
faces for im  tumblr_nn4u2dt8VU1qio5dfo1_1280.jpg  are  [[827 518 226 226]
 [778 362 308 308]
 [391 274  55  55]]
faces for im 